# Solution: The Hallucinating RAG

This notebook provides the complete solution to the debug drill.

---

In [ ]:
import re
from typing import Dict, List, Tuple

In [ ]:
# Knowledge base with CORRECT information
knowledge_base = {
    "refund_policy": {
        "title": "Refund Policy",
        "content": "We offer full refunds within 30 days of purchase. After 30 days, only store credit is available."
    }
}

context = knowledge_base['refund_policy']['content']
query = "What's your refund policy?"

# Hallucinated response (wrong numbers)
hallucinated = "You have 60 days to request a full refund. After that, you can still get 50% back."

In [ ]:
# ===== SOLUTION: Faithfulness checker =====

def check_number_faithfulness(response: str, context: str) -> Tuple[bool, List[str]]:
    """
    Check if numbers in the response appear in the context.
    """
    response_numbers = set(re.findall(r'\d+', response))
    context_numbers = set(re.findall(r'\d+', context))
    
    hallucinated = response_numbers - context_numbers
    is_faithful = len(hallucinated) == 0
    return is_faithful, list(hallucinated)

# Test on hallucinated response
is_faithful, issues = check_number_faithfulness(hallucinated, context)
print(f"Hallucinated response faithful? {is_faithful}")
print(f"Hallucinated numbers: {issues}")

In [ ]:
# ===== SOLUTION: Strong prompt =====

def strong_rag_prompt(query: str, context: str) -> str:
    """Strong prompt that reduces hallucination."""
    return f"""You are a support agent. Answer questions using ONLY the information provided below.

IMPORTANT RULES:
1. Only use facts from the CONTEXT below
2. Do NOT add information that isn't explicitly stated
3. If the context doesn't contain the answer, say "I don't have that information"
4. Quote specific numbers and policies exactly as stated

CONTEXT:
{context}

QUESTION: {query}

ANSWER (using only the context above):"""

print("=== Strong Prompt ===")
print(strong_rag_prompt(query, context))

In [ ]:
# ===== SOLUTION: Faithful response =====

def simulate_faithful_llm(query: str, context: str) -> str:
    """Simulate an LLM that only uses provided context."""
    if "30 days" in context:
        return "According to our policy, we offer full refunds within 30 days of purchase. After 30 days, only store credit is available."
    return "Based on the information provided: " + context

faithful_response = simulate_faithful_llm(query, context)
print(f"Faithful response: {faithful_response}")

# Verify
is_faithful, issues = check_number_faithfulness(faithful_response, context)
print(f"\nFaithful? {is_faithful}")

In [ ]:
# ===== SOLUTION: Response validation pipeline =====

def validate_response(response: str, context: str) -> Dict:
    """Validate that response is faithful to context."""
    result = {
        'response': response,
        'checks': []
    }
    
    # Check 1: Number faithfulness
    is_faithful, hallucinated = check_number_faithfulness(response, context)
    result['checks'].append({
        'name': 'number_faithfulness',
        'passed': is_faithful,
        'issues': hallucinated
    })
    
    # Check 2: No uncertain specifics
    uncertain_specific = bool(re.search(r'(I think|probably|maybe).*\d+', response))
    result['checks'].append({
        'name': 'no_uncertain_specifics',
        'passed': not uncertain_specific,
        'issues': ['Combines uncertainty with specific numbers'] if uncertain_specific else []
    })
    
    result['all_passed'] = all(c['passed'] for c in result['checks'])
    return result

print("=== Validation Results ===")
print("\nHallucinated:")
v1 = validate_response(hallucinated, context)
for c in v1['checks']:
    print(f"  {'✓' if c['passed'] else '❌'} {c['name']}")

print("\nFaithful:")
v2 = validate_response(faithful_response, context)
for c in v2['checks']:
    print(f"  {'✓' if c['passed'] else '❌'} {c['name']}")

## Solution Summary

**Problem:** RAG bot says "60 days" when policy says "30 days"

**Solution:**
1. **Strong prompt** with explicit rules: "Only use facts from context"
2. **Faithfulness checker** to verify numbers match context
3. **Validation pipeline** to catch hallucinations before responding

**Key techniques:**
- Number extraction with regex
- Set comparison to find hallucinated numbers
- Multiple validation checks